[Specifying shader URL, with uniforms](https://docs.google.com/document/d/1QvCvey8HaUi6SP-p36ZVwh2NGqzB_cqU_KHrAj8yFtY/edit)

[Binder notebook update](http://mybinder.org/status/rsargent/notebooks)

Saved (10): Tues 10/11 11:28am

In [10]:
import array, gzip, StringIO, struct, urllib, urllib2

def benchmark(label, data):
    zipped = StringIO.StringIO()
    zipper = gzip.GzipFile(fileobj = zipped, mode='wb')
    zipper.write(data)
    zipper.close()
    print "%.2fM / %.2fM: %s" % (len(zipped.getvalue()) / 1e6, len(data) / 1e6, label)

def encode_field(fieldspec, value):
    if 'min' in fieldspec and 'max' in fieldspec:
        value = (float(value) - fieldspec['min']) / (fieldspec['max'] - fieldspec['min'])
    if fieldspec['type'] == 'float':
        return struct.pack('<f', value)
    elif fieldspec['type'] == 'fix2':
        return struct.pack('<H', min(0xffff, max(0, int(round(value * 0xffff)))))
    elif fieldspec['type'] == 'fix3':
        return struct.pack('<I', min(0xffffff, max(0, int(round(value * 0xffffff)))))[0:3]
    else:
        raise Exception('encode_field giving up')

def decode_field(fieldspec, stream):
    if fieldspec['type'] == 'float':
        value = struct.unpack('<f', stream.read(4))[0]
    elif fieldspec['type'] == 'fix2':
        value = struct.unpack('<H', stream.read(2))[0] / float(0xffff)
    elif fieldspec['type'] == 'fix3':
        value = struct.unpack('<I', stream.read(3) + '\000')[0] / float(0xffffff)
    else:
        raise Exception('encode_field giving up')
    if 'min' in fieldspec and 'max' in fieldspec:
        value = value * (fieldspec['max'] - fieldspec['min']) + (value - fieldspec['min'])
    return value

def field_length(fieldspec):
    return len(encode_field(fieldspec, 0))

def spec_length(spec):
    ret = 0
    for fieldspec in spec['fields']:
        ret += field_length(fieldspec)
    return ret
        
def decode_row_major(spec, stream, stream_length):
    nfields = len(spec['fields'])
    if stream_length % spec_length(spec) != 0:
        raise Exception('streamlength %d should be multiple of spec_length %d' % (stream_length, spec_length(spec)))
    ret = []
    for r in range(0, stream_length / spec_length(spec)):
        for c in range(0, nfields):
            ret.append(decode_field(spec['fields'][c], stream))
    return ret

def encode_row_major(spec, data):
    nfields = len(spec['fields'])
    if len(data) % nfields != 0:
        raise Exception('Data length should be a multiple of nfields')
    ret = []
    for r in range(0, len(data), nfields):
        for c in range(0, nfields):
            ret.append(encode_field(spec['fields'][c], data[r + c]))
    return ''.join(ret)

def encode_col_major(spec, data):
    nfields = len(spec['fields'])
    if len(data) % nfields != 0:
        raise Exception('Data length should be a multiple of nfields')
    ret = []
    for c in range(0, nfields):
        for r in range(0, len(data), nfields):
            ret.append(encode_field(spec['fields'][c], data[r + c]))
    return ''.join(ret)

def encode_col_major_delta(spec, data):
    nfields = len(spec['fields'])
    if len(data) % nfields != 0:
        raise Exception('Data length should be a multiple of nfields')
    ret = []
    for c in range(0, nfields):
        for r in range(0, len(data), nfields):
            if r == 0:
                ret.append(encode_field(spec['fields'][c], data[r + c]))
            else:
                ret.append(encode_field(spec['fields'][c], data[r + c] - data[r - nfields + c]))
    return ''.join(ret)

def benchmark_encoding(label, spec, data):
    benchmark(label + ' row-major', encode_row_major(spec, data))
    benchmark(label + ' col-major', encode_col_major(spec, data))
    benchmark(label + ' col-major-delta', encode_col_major_delta(spec, data))
    

In [11]:
xyz_time_bin = open('viirs-2014.timev').read()
benchmark("xyz_time float row-major", xyz_time_bin)
xyz_time = array.array('f', xyz_time_bin)

18.41M / 26.06M: xyz_time float row-major


In [12]:
lonlat_time_temp_bin = urllib2.urlopen('https://data.cmucreatelab.org/createlab-viirs/createlab-viirs-uncorrected-20140314-20150403.bin').read()
benchmark("lonlat_time_temp float row-major", lonlat_time_temp_bin)
lonlat_time_temp = array.array('f', lonlat_time_temp_bin)

15.16M / 26.06M: lonlat_time_temp float row-major


In [14]:
lonlat_time = []
for i in range(0, len(lonlat_time_temp), 4):
    lonlat_time.append(lonlat_time_temp[i + 0])
    lonlat_time.append(lonlat_time_temp[i + 1])
    lonlat_time.append(lonlat_time_temp[i + 2])
lonlat_time_bin = array.array('f', lonlat_time).tostring()
benchmark("lonlat_time float row-major", lonlat_time_bin)

11.30M / 19.54M: lonlat_time float row-major


In [15]:
spec_fff = { 
    'stride': 12,
    'fields':[{'name':'lon', 'type':'float'},
              {'name':'lat', 'type':'float'},
              {'name':'time', 'type':'float'}]}

benchmark_encoding('lonlat_time fff', spec_fff, lonlat_time)

11.30M / 19.54M: lonlat_time fff row-major
11.12M / 19.54M: lonlat_time fff col-major
10.57M / 19.54M: lonlat_time fff col-major-delta


In [16]:
# lat and lon encoded as 24-bit fixed point, ~2m resolution
# (radius of earth * 2 * pi) / (2^24)
# time encoded as day number since posix epoch

spec_332 = { 
    'stride': 8,
    'fields':[{'name':'lon', 'type':'fix3', 'min':-180, 'max':180},
              {'name':'lat', 'type':'fix3', 'min':-90, 'max':90},
              {'name':'time', 'type':'fix2', 'min':0, 'max':5662224000}]}

benchmark_encoding('lonlat_time 332', spec_332, lonlat_time)

4.94M / 13.03M: lonlat_time 332 row-major
4.36M / 13.03M: lonlat_time 332 col-major
8.61M / 13.03M: lonlat_time 332 col-major-delta


In [ ]:
# Testing

foo = encode_row_major(spec_fff, lonlat_time[0:12])
print lonlat_time[0:12]
print decode_row_major(spec_fff, StringIO.StringIO(foo), len(foo))

foo = encode_row_major(spec_332, lonlat_time[0:12])
print len(foo)
print decode_row_major(spec_332, StringIO.StringIO(foo), len(foo))

field_spec = {'name':'foo', 'type':'fix2', 'min':0, 'max':5662224000}
f = encode_field(field_spec, 0)
print decode_field(field_spec, StringIO.StringIO(f))
f = encode_field(field_spec, 84600)
print " ".join("{:02x}".format(ord(c)) for c in f)
print decode_field(field_spec, StringIO.StringIO(f))